In [1]:
import numpy as np
from numpy import genfromtxt
import matplotlib.pyplot as plt

### Questão 1)

#### Geral

In [2]:
data_cancer = genfromtxt('/content/drive/MyDrive/AM/breastcancer.csv', delimiter = ',')

In [3]:
np.random.seed(120)

In [4]:
np.random.shuffle(data_cancer)

#### Regressão Logistica

In [5]:
def gradient_descent(X, y, alpha, epocas):

  w = np.zeros(X.shape[1])
  mse = np.zeros(epocas)

  linhas = X.shape[0]

  for t in range(epocas):
    y_pred = X @ w
    erro = y - y_pred
    mse[t] = np.mean(erro ** 2)
    w = w + (alpha/linhas) * (erro @ X)

  plt.figure(figsize=(5, 3))
  plt.plot(mse)
  plt.show()
  print("W: ", w)
  print("MSE: ", mse[-1])
  
  return w, mse

In [6]:
def normalize(X_treino, X_teste):

    Xmax = np.max(X_treino, axis = 0)
    Xmin = np.min(X_treino, axis = 0)

    X_treino_normalizado = (X_treino - Xmin)/(Xmax - Xmin)
    X_teste_normalizado = (X_teste - Xmin)/(Xmax - Xmin)

    return X_treino_normalizado, X_teste_normalizado

In [7]:
def sigmoid(z):
  return 1/(1 + np.exp(-z))

In [8]:
def logistic_regression_sgd(X_treino, X_teste, y_treino, y_teste, alpha, epocas):

  n_treino = X_treino.shape[0]
  n_teste = X_teste.shape[0]
  
  X_treino_normalizado, X_teste_normalizado = normalize(X_treino, X_teste)

  X_treino_N = np.c_[np.ones((n_treino,1)), X_treino_normalizado]
  X_teste_N = np.c_[np.ones((n_teste,1)), X_teste_normalizado]

  w = np.zeros(X_treino_N.shape[1])

  for i in range(epocas):
    for i in np.random.permutation(y_treino.shape[0]):

      y_predito = sigmoid(X_treino_N[i] @ w)
      erro = y_treino[i] - y_predito
      w = w + alpha * erro * X_treino_N[i] 

  y_predito = sigmoid(X_teste_N @ w)
  yP = (y_predito >= 0.5).astype(int)

  return np.c_[yP, y_teste]

#### Análise do discriminante Gaussiano

In [9]:
def gaussian_discriminant_analysis(X_treino, X_teste, y_treino, y_teste):
  
  col = X_treino.shape[1]
  n_treino = X_treino.shape[0]
  n_teste = X_teste.shape[0]
  
  X_treino_normalizado, X_teste_normalizado = normalize(X_treino, X_teste)

  X_treino_a = X_treino_normalizado[y_treino == 0]
  X_treino_b = X_treino_normalizado[y_treino == 1]

  n_treino_a = X_treino_a.shape[0]
  n_treino_b = X_treino_b.shape[0]

  media_a = np.mean(X_treino_a, axis=0)
  media_b = np.mean(X_treino_b, axis=0)

  cov_a = np.cov(X_treino_a, rowvar=False, ddof=1)
  cov_b = np.cov(X_treino_b, rowvar=False, ddof=1)

  log_a = np.log(n_treino_a/n_treino)
  log_b = np.log(n_treino_b/n_treino)

  logdet_a = np.log(np.linalg.det(cov_a))
  logdet_b = np.log(np.linalg.det(cov_b))

  inv_a = np.linalg.inv(cov_a)
  inv_b = np.linalg.inv(cov_b)

  p_a = log_a - 0.5 * logdet_a - 0.5 * np.sum((X_teste_normalizado - media_a) @ inv_a * (X_teste_normalizado - media_a), axis=1)
  p_b = log_b - 0.5 * logdet_b - 0.5 * np.sum((X_teste_normalizado - media_b) @ inv_b * (X_teste_normalizado - media_b), axis=1)

  y_predito = (p_b > p_a).astype(int)
  
  return np.c_[y_predito, y_teste]

#### Naive bayes gaussiano

In [10]:
def naive_bayes_gaussiano(X_treino, X_teste, y_treino, y_teste):

  col = X_treino.shape[1]
  n_treino = X_treino.shape[0]
  n_teste = X_teste.shape[0]
  
  X_treino_normalizado, X_teste_normalizado = normalize(X_treino, X_teste)

  X_treino_a = X_treino_normalizado[y_treino == 0]
  X_treino_b = X_treino_normalizado[y_treino == 1]

  n_treino_a = X_treino_a.shape[0]
  n_treino_b = X_treino_b.shape[0]

  media_a = np.mean(X_treino_a, axis=0)
  media_b = np.mean(X_treino_b, axis=0)

  cov_a = np.cov(X_treino_a, rowvar=False)
  cov_b = np.cov(X_treino_b, rowvar=False)

  log_a = np.log(n_treino_a/n_treino)
  log_b = np.log(n_treino_b/n_treino)

  p_a = -(1/2) * np.sum(np.log(2 * np.pi * cov_a)) - (1/2) * np.sum((X_teste_normalizado - media_a) @ np.linalg.inv(cov_a) * (X_teste_normalizado - media_a), axis=1) + log_a
  p_b = -(1/2) * np.sum(np.log(2 * np.pi * cov_b)) - (1/2) * np.sum((X_teste_normalizado - media_b) @ np.linalg.inv(cov_b) * (X_teste_normalizado - media_b), axis=1) + log_b

  y_predito = (p_b > p_a).astype(int)

  return np.c_[y_predito, y_teste]

#### Resultados

In [11]:
k = 10
n = data_cancer.shape[0]
s = n // k
resto = n % k
indices = np.arange(n)
np.random.shuffle(indices)

treino = []
teste = []
inicio = 0

for i in range(k):
    fim = inicio + s + (resto > 0)
    resto -= 1
    test_indices = indices[inicio:fim]
    train_indices = np.concatenate((indices[:inicio], indices[fim:]))
    treino.append(data_cancer[train_indices])
    teste.append(data_cancer[test_indices])
    inicio = fim

In [15]:
rl = []
nbg = []
agd = []

alpha = 0.01
epocas = 100

for i in range(k):
  tr = treino[i]
  te = teste[i]
  X_treino = tr[: , :-1]
  X_teste = te[: , :-1]
  y_treino = tr[: ,-1]
  y_teste = te[: ,-1]

  rl.append(logistic_regression_sgd(X_treino, X_teste, y_treino, y_teste, alpha, epocas))
  agd.append(gaussian_discriminant_analysis(X_treino, X_teste, y_treino, y_teste))
  nbg.append(naive_bayes_gaussiano(X_treino, X_teste, y_treino, y_teste))

<ipython-input-10-51cfac0d9893>:24: RuntimeWarning: invalid value encountered in log
  p_a = -(1/2) * np.sum(np.log(2 * np.pi * cov_a)) - (1/2) * np.sum((X_teste_normalizado - media_a) @ np.linalg.inv(cov_a) * (X_teste_normalizado - media_a), axis=1) + log_a
<ipython-input-10-51cfac0d9893>:25: RuntimeWarning: invalid value encountered in log
  p_b = -(1/2) * np.sum(np.log(2 * np.pi * cov_b)) - (1/2) * np.sum((X_teste_normalizado - media_b) @ np.linalg.inv(cov_b) * (X_teste_normalizado - media_b), axis=1) + log_b


In [17]:
for i in range(k):
  print("- Fold: ", i+1)

  LR = rl[i]
  ADG = agd[i]
  NBG = nbg[i]
  acuracia_lr = np.sum(LR[:, 0] == LR[:, 1])/LR.shape[0]
  print("1) Regressão Logística:")
  print("Acurácia: ", acuracia_lr)
  acuracia_adg = np.sum(ADG[:, 0] == ADG[:, 1])/ADG.shape[0]
  print("2) Análise do discriminante Gaussiano: ")
  print("Acurácia: ", acuracia_adg)
  acucaria_ngb = np.sum(NBG[:, 0] == NBG[:, 1])/NBG.shape[0]
  print("3) Naive Bayes Gaussiano: ")
  print("Acurácia: ", acucaria_ngb)

- Fold:  1
1) Regressão Logística:
Acurácia:  0.9473684210526315
2) Análise do discriminante Gaussiano: 
Acurácia:  0.9122807017543859
3) Naive Bayes Gaussiano: 
Acurácia:  0.7368421052631579
- Fold:  2
1) Regressão Logística:
Acurácia:  0.9649122807017544
2) Análise do discriminante Gaussiano: 
Acurácia:  0.9473684210526315
3) Naive Bayes Gaussiano: 
Acurácia:  0.5614035087719298
- Fold:  3
1) Regressão Logística:
Acurácia:  0.9649122807017544
2) Análise do discriminante Gaussiano: 
Acurácia:  0.9649122807017544
3) Naive Bayes Gaussiano: 
Acurácia:  0.6491228070175439
- Fold:  4
1) Regressão Logística:
Acurácia:  0.9298245614035088
2) Análise do discriminante Gaussiano: 
Acurácia:  0.9298245614035088
3) Naive Bayes Gaussiano: 
Acurácia:  0.5263157894736842
- Fold:  5
1) Regressão Logística:
Acurácia:  0.9824561403508771
2) Análise do discriminante Gaussiano: 
Acurácia:  0.9824561403508771
3) Naive Bayes Gaussiano: 
Acurácia:  0.6491228070175439
- Fold:  6
1) Regressão Logística:
Acurá

### Questão 2)

#### Geral

In [18]:
data_vehicle = genfromtxt('/content/drive/MyDrive/AM/vehicle.csv', delimiter = ',')

In [19]:
np.random.seed(120)

In [20]:
np.random.shuffle(data_vehicle)

#### Regressão Logística com softmax

In [21]:
def softmax(z):
  return (np.exp(z)/sum(np.exp(z)))

In [23]:
def logistic_regression_sgd(X_treino, X_teste, y_treino, y_teste, alpha, epocas):

  n_treino = X_treino.shape[0]
  n_teste = X_teste.shape[0]
  
  X_treino_normalizado, X_teste_normalizado = normalize(X_treino, X_teste)

  X_treino_N = np.c_[np.ones((n_treino,1)), X_treino_normalizado]
  X_teste_N = np.c_[np.ones((n_teste,1)), X_teste_normalizado]

  w = np.zeros(X_treino_N.shape[1])

  for i in range(epocas):
    for i in np.random.permutation(y_treino.shape[0]):

      y_predito = softmax(X_treino_N[i] @ w)
      erro = y_treino[i] - y_predito
      w = w + alpha * erro * X_treino_N[i] 

  y_predito = softmax(X_teste_N @ w)
  yP = (y_predito >= 0.5).astype(int)

  return np.c_[yP, y_teste]

#### Resultados

In [28]:
k = 10
n = data_vehicle.shape[0]
s = n // k
resto = n % k
indices = np.arange(n)
np.random.shuffle(indices)

treino = []
teste = []
inicio = 0

for i in range(k):
    fim = inicio + s + (resto > 0)
    resto -= 1
    test_indices = indices[inicio:fim]
    train_indices = np.concatenate((indices[:inicio], indices[fim:]))
    treino.append(data_vehicle[train_indices])
    teste.append(data_vehicle[test_indices])
    inicio = fim

In [30]:
#rl = []
nbg = []
agd = []

alpha = 0.01
epocas = 100

for i in range(k):
  tr = treino[i]
  te = teste[i]
  X_treino = tr[: , :-1]
  X_teste = te[: , :-1]
  y_treino = tr[: ,-1]
  y_teste = te[: ,-1]

  #rl.append(logistic_regression_sgd(X_treino, X_teste, y_treino, y_teste, alpha, epocas))
  agd.append(gaussian_discriminant_analysis(X_treino, X_teste, y_treino, y_teste))
  nbg.append(naive_bayes_gaussiano(X_treino, X_teste, y_treino, y_teste))

<ipython-input-10-51cfac0d9893>:24: RuntimeWarning: invalid value encountered in log
  p_a = -(1/2) * np.sum(np.log(2 * np.pi * cov_a)) - (1/2) * np.sum((X_teste_normalizado - media_a) @ np.linalg.inv(cov_a) * (X_teste_normalizado - media_a), axis=1) + log_a
<ipython-input-10-51cfac0d9893>:25: RuntimeWarning: invalid value encountered in log
  p_b = -(1/2) * np.sum(np.log(2 * np.pi * cov_b)) - (1/2) * np.sum((X_teste_normalizado - media_b) @ np.linalg.inv(cov_b) * (X_teste_normalizado - media_b), axis=1) + log_b


In [31]:
for i in range(k):
  print("- Fold: ", i+1)

  #LR = rl[i]
  ADG = agd[i]
  NBG = nbg[i]
  #acuracia_lr = np.sum(LR[:, 0] == LR[:, 1])/LR.shape[0]
  #print("1) Regressão Logística:")
  #print("Acurácia: ", acuracia_lr)
  acuracia_adg = np.sum(ADG[:, 0] == ADG[:, 1])/ADG.shape[0]
  print("2) Análise do discriminante Gaussiano: ")
  print("Acurácia: ", acuracia_adg)
  acucaria_ngb = np.sum(NBG[:, 0] == NBG[:, 1])/NBG.shape[0]
  print("3) Naive Bayes Gaussiano: ")
  print("Acurácia: ", acucaria_ngb)

- Fold:  1
2) Análise do discriminante Gaussiano: 
Acurácia:  0.5058823529411764
3) Naive Bayes Gaussiano: 
Acurácia:  0.23529411764705882
- Fold:  2
2) Análise do discriminante Gaussiano: 
Acurácia:  0.5176470588235295
3) Naive Bayes Gaussiano: 
Acurácia:  0.27058823529411763
- Fold:  3
2) Análise do discriminante Gaussiano: 
Acurácia:  0.5647058823529412
3) Naive Bayes Gaussiano: 
Acurácia:  0.3058823529411765
- Fold:  4
2) Análise do discriminante Gaussiano: 
Acurácia:  0.4588235294117647
3) Naive Bayes Gaussiano: 
Acurácia:  0.27058823529411763
- Fold:  5
2) Análise do discriminante Gaussiano: 
Acurácia:  0.5647058823529412
3) Naive Bayes Gaussiano: 
Acurácia:  0.3058823529411765
- Fold:  6
2) Análise do discriminante Gaussiano: 
Acurácia:  0.4235294117647059
3) Naive Bayes Gaussiano: 
Acurácia:  0.2
- Fold:  7
2) Análise do discriminante Gaussiano: 
Acurácia:  0.5119047619047619
3) Naive Bayes Gaussiano: 
Acurácia:  0.21428571428571427
- Fold:  8
2) Análise do discriminante Gaussi